
## Коллаборативная фильтрация

### Преподаватель: Александр Кондрашкин
#### ПАКЕТ SURPRISE

− используйте данные MovieLens 1M  
− можно использовать любые модели из пакета  
− получите RMSE на тестовом сете 0.87 и ниже  

#### Решением будет ссылка на гитхаб с готовым ноутбуком

In [2]:
from surprise import KNNWithMeans, KNNBasic,KNNWithZScore,KNNBaseline,CoClustering,BaselineOnly,SlopeOne,NMF,SVDpp,SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

import random

from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [3]:
movies = pd.read_csv('../ml-1m/movies.dat', sep='::', names=['movieId', 'title', 'genres'])
ratings = pd.read_csv('../ml-1m/ratings.dat', sep='::', names=['userId', 'movieId', 'rating', 'timestamp'])

/home/gavril/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/gavril/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
movies.shape

(3883, 3)

In [5]:
ratings.shape

(1000209, 4)

In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings[movies_with_ratings.userId == 1.0].title.unique()

array(['Toy Story (1995)', 'Pocahontas (1995)', 'Apollo 13 (1995)',
       'Star Wars: Episode IV - A New Hope (1977)',
       "Schindler's List (1993)", 'Secret Garden, The (1993)',
       'Aladdin (1992)', 'Snow White and the Seven Dwarfs (1937)',
       'Beauty and the Beast (1991)', 'Fargo (1996)',
       'James and the Giant Peach (1996)',
       'Wallace & Gromit: The Best of Aardman Animation (1996)',
       'Close Shave, A (1995)', 'Hunchback of Notre Dame, The (1996)',
       'My Fair Lady (1964)', 'Wizard of Oz, The (1939)', 'Gigi (1958)',
       'Cinderella (1950)', 'Mary Poppins (1964)', 'Dumbo (1941)',
       'Sound of Music, The (1965)', 'E.T. the Extra-Terrestrial (1982)',
       "One Flew Over the Cuckoo's Nest (1975)",
       'Princess Bride, The (1987)', 'To Kill a Mockingbird (1962)',
       'Dead Poets Society (1989)', 'Back to the Future (1985)',
       'Ben-Hur (1959)', 'Ponette (1996)', 'Hercules (1997)',
       'Titanic (1997)', 'Last Days of Disco, The (1998)',

In [9]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,5.0,978824268.0
1,1,Toy Story (1995),Animation|Children's|Comedy,6.0,4.0,978237008.0
2,1,Toy Story (1995),Animation|Children's|Comedy,8.0,4.0,978233496.0
3,1,Toy Story (1995),Animation|Children's|Comedy,9.0,5.0,978225952.0
4,1,Toy Story (1995),Animation|Children's|Comedy,10.0,5.0,978226474.0


In [10]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [11]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [12]:
dataset.shape

(1000209, 3)

In [13]:
#x.isnull().any()
#x.var_  = x.var_.fillna(0)

In [14]:
dataset.isnull().any()

uid       False
iid       False
rating    False
dtype: bool

In [15]:
ratings.rating.min()

1

In [16]:
ratings.rating.max()

5

In [17]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

In [18]:
data

In [79]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

### Обучим какую-нибдуь модельку

In [80]:
algo_KNNWithMeans = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo_KNNWithMeans.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [81]:
test_pred =algo_KNNWithMeans.test(testset)

In [82]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9021


0.9021257504993911

####  Гм, ну слабенько пока. Будем пытаться улчушить

### Попробуем подобрать параметры
#### Там какая-то ерунда  нельзя просто взять и запихнуть результаты train_test_split-a в GridSearchCV ,в FAQ surpris-a нашел что можно делать так

In [20]:
%%time
params = {
    'k':[2,10,50,60,80,100,150,200],
    'sim_options': {'name': ['msd', 'cosine','pearson_baseline'],
                              'min_support': [1, 5],
                              'user_based': [False,True]}
}



CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [21]:
%%time
# Load the full dataset.
##algo = grid_search.best_estimator['rmse']data = Dataset.load_builtin('ml-100k')
raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')
#param_grid = {'k':[2,10]}
grid_search = GridSearchCV(KNNWithMeans, params, measures=['rmse'], cv=3)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Grid Search...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity m

Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computi

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarit

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [78]:
## Ну вот грид предлагает таки user-based подход  не использовать
print(grid_search.best_params['rmse'])

{'k': 50, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}


### Обучим еще пучок моделей, может быть получше найдется

In [65]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

In [66]:
%%time
algo_KNNBasic= KNNBasic()
algo_KNNBasic.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 14 s, sys: 256 ms, total: 14.2 s
Wall time: 14.2 s


In [67]:
%%time
algo_KNNWithZScore= KNNWithZScore()
algo_KNNWithZScore.fit(trainset)


Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 14.5 s, sys: 244 ms, total: 14.8 s
Wall time: 14.7 s


In [68]:
%%time
algo_KNNBaseline= KNNBaseline()
algo_KNNBaseline.fit(trainset)



Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
CPU times: user 15 s, sys: 236 ms, total: 15.2 s
Wall time: 15.2 s


In [69]:
%%time
algo_CoClustering= CoClustering()
algo_CoClustering.fit(trainset)





CPU times: user 11.4 s, sys: 20 ms, total: 11.4 s
Wall time: 11.4 s


In [70]:
%%time
algo_BaselineOnly= BaselineOnly()
algo_BaselineOnly.fit(trainset)


Estimating biases using als...
CPU times: user 999 ms, sys: 8 µs, total: 999 ms
Wall time: 1 s


In [71]:
%%time
algo_SlopeOne= SlopeOne()
algo_SlopeOne.fit(trainset)


CPU times: user 10.4 s, sys: 120 ms, total: 10.6 s
Wall time: 10.5 s


In [72]:
%%time
algo_NMF= NMF()
algo_NMF.fit(trainset)


CPU times: user 28.1 s, sys: 3.96 ms, total: 28.1 s
Wall time: 28.1 s


In [73]:
%%time
algo_SVD= SVD()
algo_SVD.fit(trainset)


CPU times: user 25.2 s, sys: 0 ns, total: 25.2 s
Wall time: 25.2 s


In [74]:
%%time
algo_SVDpp= SVDpp()
algo_SVDpp.fit(trainset)


CPU times: user 23min 8s, sys: 43.8 ms, total: 23min 8s
Wall time: 23min 8s


In [75]:
models=['SVD','SVDpp','NMF','SlopeOne','BaselineOnly','CoClustering','KNNBaseline','KNNWithZScore','KNNBasic']

In [76]:
for model_ in models:
    print(model_,'---','root_mean_squared_error = ', accuracy.rmse(eval('algo_'+model_).test(testset), verbose=False))

SVD --- root_mean_squared_error =  0.8925986446848684
SVDpp --- root_mean_squared_error =  0.8766763641250948
NMF --- root_mean_squared_error =  0.9246364979058038
SlopeOne --- root_mean_squared_error =  0.9111621034977191
BaselineOnly --- root_mean_squared_error =  0.9129375056924909
CoClustering --- root_mean_squared_error =  0.9202193613500261
KNNBaseline --- root_mean_squared_error =  0.9031929084897661
KNNWithZScore --- root_mean_squared_error =  0.9372808247105481
KNNBasic --- root_mean_squared_error =  0.936783477832214


### Ну видно, что без настройки лучший результата дает SVD++ (но это немножко другой класс моделей), а так -KNNBaseline
###  Попробуем для него подобрать лучшие парметры

In [21]:
%%time
# Load the full dataset.
##algo = grid_search.best_estimator['rmse']data = Dataset.load_builtin('ml-100k')
raw_ratings = data.raw_ratings

# shuffle ratings if you want
random.shuffle(raw_ratings)

# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # data is now the set A

# Select your best algo with grid search.
print('Grid Search...')
#param_grid = {'k':[2,10]}
grid_search_KNNBaseline = GridSearchCV(KNNBaseline, params, measures=['rmse'], cv=3)
grid_search_KNNBaseline.fit(data)


Grid Search...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity m

Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computin

Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimatin

NameError: name 'grid_search' is not defined

In [22]:

algo = grid_search_KNNBaseline.best_estimator['rmse']

# retrain on the whole set A
trainset = data.build_full_trainset()
algo.fit(trainset)

# Compute biased accuracy on A
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on B
testset = data.construct_testset(B_raw_ratings)  # testset is now the set B
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Biased accuracy on A,   RMSE: 0.5640
Unbiased accuracy on B, RMSE: 0.8596


0.8596469833756718

### Чуть-чуть похуже реузультат получился, чем KNNWithMeans. Ктати, видимо сильно переобучается 

In [23]:
# Тоже user-based  не выбрал
print(grid_search_KNNBaseline.best_params['rmse'])

{'k': 50, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}
